In [3]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm
from LEEP import LEEP
from LogME import LogME


In [4]:
from keras.utils import np_utils
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.datasets import cifar10


def load_cifar10_data(img_rows, img_cols):

    # Load cifar10 training and validation sets
    (X_train, Y_train), (X_valid, Y_valid) = cifar10.load_data()
    nb_train_samples = 40000 
    nb_valid_samples = 10000 
    num_classes = 10

    # Resize trainging images

    X_train = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_train[:nb_train_samples,:,:,:]])
    X_valid = np.array([cv2.resize(img, (img_rows,img_cols)) for img in X_valid[:nb_valid_samples,:,:,:]])

    # Transform targets to keras compatible format
    Y_train = np_utils.to_categorical(Y_train[:nb_train_samples], num_classes)
    Y_valid = np_utils.to_categorical(Y_valid[:nb_valid_samples], num_classes)

    return X_train, Y_train, X_valid, Y_valid

In [5]:
X_train, Y_train, X_valid, Y_valid=load_cifar10_data(32, 32)

In [6]:
from tensorflow.keras import layers
from keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers

tf.keras.backend.set_floatx('float64')  

class Model1(tf.keras.Model):
    def __init__(self,feature_layer,**kwargs):
        super().__init__()
        self.d1=feature_layer
        # self.d2=layers.Dropout(0.25)
        self.d5=layers.Dense(10,activation='softmax')

    def forward(self, x,train=False):
        x = self.d1(x)[:,0,0,:]
        # x=self.d2(x)
        x = self.d5(x)        
        return x

    def call(self, inputs):
        x = self.forward(inputs,train=False)
        return x     



# from tensorflow.keras.applications.resnet import ResNet50
# base_model = ResNet50(weights='imagenet', include_top=True,input_tensor=keras.layers.Input(shape=(32, 32, 3))) ### probablistic predictions, required for LEEP

# from tensorflow.keras.applications.efficientnet import EfficientNetB0
# base_model = EfficientNetB0(weights='imagenet', include_top=True,input_tensor=keras.layers.Input(shape=(32, 32, 3))) ### probablistic predictions, required for LEEP

# from tensorflow.keras.applications.xception import Xception
# base_model = Xception(weights='imagenet', include_top=True,input_tensor=keras.layers.Input(shape=(32, 32, 3))) ### probablistic predictions, required for LEEP




# from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
# base_model = MobileNetV2(weights='imagenet', include_top=True,input_tensor=keras.layers.Input(shape=(32, 32, 3))) ### probablistic predictions, required for LEEP

from tensorflow.keras.applications.densenet import DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=True,input_tensor=keras.layers.Input(shape=(32, 32, 3))) ### probablistic predictions, required for LEEP


base_model_emb = tf.keras.Model(inputs=base_model.inputs, outputs=base_model.layers[-2].output)  ### Gives embedding

33202176/33188688 [==============================] - 1s 0us/step


In [7]:
def compute_neural_collapse_score(E,L):
   Ind=np.unique(L)
   class_means=[]
   
   for i in Ind:  
     ind=np.where(L==i)[0]
     data=E[ind,:]
     mean=np.mean(data,axis=0)
     class_means.append(mean)
     
   num=0 
   for i in range(0,len(E)):
       a=np.linalg.norm(E[i]-class_means[int(L[i])]) 
       num=num+a
   N=num/len(E)


   num=0 
   sum=0
   for i in range(0,len(class_means)):
       c1=class_means[i]
       for j in range(0,len(class_means)):
           c2=class_means[j]
           if i!=j:
              sum=sum+np.linalg.norm(c1-c2) 
              num=num+1
   D=sum/num
   score=N/D
    
   return score

In [8]:
def get_embedding(X_train,Y_train,model): 
    X=np.array_split(X_train,50)
    Y=np.array_split(Y_train,50)
    E=[]
    L=[]

    for i in tqdm(range(len(X))):
        e=model(X[i])
        E.append(e)
        L.append(Y[i])
     
    E=np.vstack(E)
    L=np.vstack(L)
    return E, L


In [9]:
emb, label= get_embedding(X_train,Y_train,base_model_emb)
L1=np.argmax(label,axis=1)
nc = compute_neural_collapse_score(emb,L1)
print(nc)

100%|██████████| 50/50 [01:25<00:00,  1.72s/it]


2.273530291811587


In [10]:
logme = LogME(regression=False)
score=logme.fit(emb,L1)
print(score)

-0.07839117674345226


In [11]:
preds, label= get_embedding(X_train,Y_train,base_model)
L1=np.argmax(label,axis=1)
leep = LEEP(preds,L1)
print(leep)

100%|██████████| 50/50 [01:22<00:00,  1.65s/it]


-2.078625739711616
